In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn import decomposition
from xgboost import XGBClassifier
import pickle

In [0]:
def featurize(dummies):
  pca = decomposition.PCA(n_components=2)
  pca.fit(dummies)
  X_PCA = pca.transform(dummies)
  return X_PCA

In [0]:
def datarize2(path):
  df_rows = []
  with open(path, encoding="utf-8") as f:
    for line in f.readlines():
      line = line.strip()
      features = line.split("\x01")
      df_rows.append(features)

  df = pd.DataFrame(df_rows)
  columns = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
              "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
              "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
              "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
              "enaging_user_account_creation", "engagee_follows_engager"]
  df.columns = columns
  return df

In [0]:
def encoding2(df):
  df.loc[(df['present_media']!='Photo') & (df['present_media']!='Video'), 'present_media']='others'      
  df.loc[(df['tweet_type']!='Top_Level') & (df['tweet_type']!='Retweet') & (df['tweet_type']!='Quote'), 'tweet_type'] = 'Top_Level'   
  df.loc[(df['language']!='D3164C7FBCF2565DDF915B1B3AEFB1DC') & (df['language']!='22C448FF81263D4BAF2A176145EE9EAD') & (df['language']!='06D61DCBBE938971E1EA0C38BD9B5446') & (df['language']!='ECED8A16BE2A5E8871FD55F4842F16B1') & (df['language']!='B9175601E87101A984A50F8A62A1C374'), 'language'] = 'others'
  df = df.drop(columns=['text_tokens', 'hashtags', 'tweet_id', 'present_links', 'present_domains', 'tweet_timestamp', 'enaging_user_id', 'engaged_with_user_account_creation', 'engaged_with_user_id', 'enaging_user_account_creation'])

  dummies = pd.get_dummies(
      df, 
      columns=['language', 'engaged_with_user_is_verified', 'enaging_user_is_verified', 'engagee_follows_engager', 'present_media', 'tweet_type'],
      prefix=['language', 'engaged_with_user_is_verified', 'enaging_user_is_verified', 'engagee_follows_engager', 'present_media', 'tweet_type'],
      prefix_sep="_",
      dummy_na=False,
      drop_first=False
  )
  return dummies

In [0]:
reply_xgbc = pickle.load(open("/content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/reply_xgbc.pickle.dat", "rb"))

In [0]:
path = '/content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/validation data'
filepaths = [os.path.join(path,fileName) for fileName in  os.listdir(path)[2:]]

In [0]:
reply_pred = []

In [8]:
for filepath in filepaths:
  df = datarize2(filepath)
  dummies = encoding2(df)
  X_PCA = featurize(dummies)
  y_pred = reply_xgbc.predict_proba(X_PCA)[:,1]
  result_df = df[['tweet_id', 'enaging_user_id']]
  result_df['Prediction'] = y_pred
  result_df.columns = ['Tweet_Id', 'User_Id', 'Prediction']
  reply_pred.append(result_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [0]:
reply_predection = pd.concat(reply_pred)

In [0]:
reply_predection.to_csv('reply_pred.csv', index=False)